In [ ]:
pip install dbfread

In [ ]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 82.8 MB/s eta 0:00:00


In [ ]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin

# Step 1: Define path
base_path = r"/content/Fault"
shapefile_path = os.path.join(base_path, "Fault.shp")

# Step 2: Load shapefile
gdf = gpd.read_file(shapefile_path)
print("Original CRS:", gdf.crs)

# Step 3: Project to UTM for 30m resolution (automatic zone)
gdf_utm = gdf.to_crs(gdf.estimate_utm_crs())
print("Projected CRS:", gdf_utm.crs)

# Step 4: Get bounds and create a 30m grid
xmin, ymin, xmax, ymax = gdf_utm.total_bounds
resolution = 30  # 30 meters

width = int((xmax - xmin) / resolution)
height = int((ymax - ymin) / resolution)

x_coords = np.linspace(xmin + resolution / 2, xmax - resolution / 2, width)
y_coords = np.linspace(ymax - resolution / 2, ymin + resolution / 2, height)
xx, yy = np.meshgrid(x_coords, y_coords)

pixel_points = np.vstack([xx.ravel(), yy.ravel()]).T

# Step 5: Calculate distance to each fault line
proximity_maps = []

for fault in gdf_utm.geometry:
    distances = np.array([fault.distance(Point(xy)) for xy in pixel_points])
    distance_image = distances.reshape((height, width))
    proximity_maps.append(distance_image)

# Step 6: Stack maps
proximity_stack = np.stack(proximity_maps, axis=0)

# Step 7: Export first proximity map as GeoTIFF
output_tif = "fault_proximity_30m.tif"
transform = from_origin(xmin, ymax, resolution, resolution)  # top-left origin

with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=proximity_stack[0].dtype,
    crs=gdf_utm.crs,
    transform=transform,
) as dst:
    dst.write(proximity_stack[0], 1)

print(f"Saved proximity map to: {output_tif}")


Original CRS: EPSG:32644
Projected CRS: EPSG:32643
Saved proximity map to: fault_proximity_30m.tif
